In [24]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from pandas import read_csv
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [2]:
df = pd.read_csv('datasets/emotics_annotations_clean_v4.csv')
df.head()

,filename,folder,image_size,original_database,person,body_bbox,annotations_categories,annotations_continuous,gender,age,...,shape_y,x1,y1,x2,y2,valence,arousal,dominance,categories,n_cat
0,COCO_val2014_000000562243.jpg,mscoco/images,"(array([[640]], dtype=uint16), array([[640]], ...","(array(['mscoco'], dtype='<U6'), array([[(arra...","(array([[ 86, 58, 564, 628]], dtype=uint16), ...",[[ 86 58 564 628]],"(array([[array(['Disconnection'], dtype='<U13'...","(array([[5]], dtype=uint8), array([[3]], dtype...",Male,Adult,...,640.0,86.0,58.0,564.0,628.0,5.0,3.0,9.0,"Disconnection,Doubt/Confusion",2.0
1,COCO_train2014_000000288841.jpg,mscoco/images,"(array([[640]], dtype=uint16), array([[480]], ...","(array(['mscoco'], dtype='<U6'), array([[(arra...","(array([[485, 149, 605, 473]], dtype=uint16), ...",[[485 149 605 473]],"(array([[array(['Anticipation'], dtype='<U12')...","(array([[6]], dtype=uint8), array([[4]], dtype...",Male,Adult,...,480.0,485.0,149.0,605.0,473.0,6.0,4.0,7.0,Anticipation,1.0
2,COCO_val2014_000000558171.jpg,mscoco/images,"(array([[640]], dtype=uint16), array([[480]], ...","(array(['mscoco'], dtype='<U6'), array([[(arra...","(array([[305, 92, 461, 465]], dtype=uint16), ...",[[305 92 461 465]],"(array([[array(['Engagement'], dtype='<U10'),\...","(array([[7]], dtype=uint8), array([[8]], dtype...",Male,Teenager,...,480.0,305.0,92.0,461.0,465.0,7.0,8.0,8.0,"Engagement,Excitement,Happiness",3.0
3,COCO_train2014_000000369575.jpg,mscoco/images,"(array([[480]], dtype=uint16), array([[640]], ...","(array(['mscoco'], dtype='<U6'), array([[(arra...","(array([[221, 63, 448, 372]], dtype=uint16), ...",[[221 63 448 372]],"(array([[array(['Aversion'], dtype='<U8'),\n ...","(array([[8]], dtype=uint8), array([[9]], dtype...",Male,Kid,...,640.0,221.0,63.0,448.0,372.0,8.0,9.0,8.0,"Aversion,Pleasure",2.0
4,COCO_train2014_000000213009.jpg,mscoco/images,"(array([[500]], dtype=uint16), array([[333]], ...","(array(['mscoco'], dtype='<U6'), array([[(arra...","(array([[ 44, 143, 150, 288]], dtype=uint16), ...",[[ 44 143 150 288]],"(array([[array(['Confidence'], dtype='<U10'),\...","(array([[7]], dtype=uint8), array([[9]], dtype...",Male,Adult,...,333.0,44.0,143.0,150.0,288.0,7.0,9.0,10.0,"Confidence,Excitement",2.0


In [7]:
# create a mapping of tags to integers given the loaded mapping file
def create_tag_mapping(mapping_csv):
	# create a set of all known tags
	labels = set()
	for i in range(len(mapping_csv)):
		# convert spaced separated tags into an array of tags
		tags = mapping_csv['categories'][i].split(',')
		# add tags to the set of known labels
		labels.update(tags)
	# convert set of labels to a list to list
	labels = list(labels)
	# order set alphabetically
	labels.sort()
	# dict that maps labels to integers, and the reverse
	labels_map = {labels[i]:i for i in range(len(labels))}
	inv_labels_map = {i:labels[i] for i in range(len(labels))}
	return labels_map, inv_labels_map

In [8]:
mapping, inv_mapping = create_tag_mapping(df)
print(len(mapping))
print(mapping)

26
{'Affection': 0, 'Anger': 1, 'Annoyance': 2, 'Anticipation': 3, 'Aversion': 4, 'Confidence': 5, 'Disapproval': 6, 'Disconnection': 7, 'Disquietment': 8, 'Doubt/Confusion': 9, 'Embarrassment': 10, 'Engagement': 11, 'Esteem': 12, 'Excitement': 13, 'Fatigue': 14, 'Fear': 15, 'Happiness': 16, 'Pain': 17, 'Peace': 18, 'Pleasure': 19, 'Sadness': 20, 'Sensitivity': 21, 'Suffering': 22, 'Surprise': 23, 'Sympathy': 24, 'Yearning': 25}


In [42]:
# create a mapping of filename to tags
def create_file_mapping(mapping_csv):
	mapping = dict()
	for i in range(len(mapping_csv)):
		name, tags = mapping_csv['filename'][i], mapping_csv['categories'][i]
		mapping[name] = tags.split(',')
	return mapping

In [11]:
# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
	# create empty vector
	encoding = zeros(len(mapping), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[mapping[tag]] = 1
	return encoding

In [49]:
# load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    # enumerate files in the directory
    for f in listdir(path):
        f = path + f + '/images/'
        for filename in listdir(f):
            # load image
            photo = load_img(f + filename, target_size=(128,128))
            # convert to numpy array
            photo = img_to_array(photo, dtype='uint8')
            # get tags
            try:
                tags = file_mapping[filename]
            except:
                print(filename)
            # one hot encode tags
            target = one_hot_encode(tags, tag_mapping)
            # store
            photos.append(photo)
            targets.append(target)
    X = asarray(photos, dtype='uint8')
    y = asarray(targets, dtype='uint8')
    return X, y

In [50]:
# load the mapping file
filename = 'datasets/emotics_annotations_clean_v4.csv'
mapping_csv = pd.read_csv(filename)
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(mapping_csv)
# load the jpeg images
folder = 'raw_data/EMOTIC_DATASET/emotic/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
# save both arrays to one file in compressed format
savez_compressed('planet_data.npz', X, y)

labelme_crjcqqxpmoznjzz.jpg
labelme_evanxtujzjwtkni.jpg
labelme_fpqubxfwifldqao.jpg
labelme_gdsdayxizczuvkj.jpg
labelme_ncycepcfbwvumsg.jpg
labelme_pwunoqiidsjpadz.jpg
labelme_sxsmqmkhxxlkoub.jpg
labelme_uacrknnpbhjfbbw.jpg
labelme_uvguzeesvhnciki.jpg
labelme_weztrxfgtyoqzvc.jpg
labelme_xolbqxbmjhejyaj.jpg
labelme_ykidtiasrnvxzkg.jpg
sun_aaaevafjexfriuyd.jpg
sun_aaflptztmjujrsgx.jpg
sun_aafnqaedispyjbis.jpg
sun_aajngarqbqcvnkzx.jpg
sun_aakdnzlwzpnymfna.jpg
sun_aaljzlbxldyceadt.jpg
sun_aapmhscnpfqopdpt.jpg
sun_aarawdireoqpzcsr.jpg
sun_aatebnvzwojqxbfw.jpg
sun_aauiyomaqbowzixw.jpg
sun_aaulrtutrjgvpnvi.jpg
sun_abjhymkavymmcgrt.jpg
sun_abmqrelrfzbreewi.jpg
sun_absuvxrxwqknadoz.jpg
sun_acjhbwdrcmcuhryu.jpg
sun_acjkowywemvcqzvz.jpg
sun_acscizucryhwlsjo.jpg
sun_acyurcvzexpfrsyb.jpg
sun_adbvfipbkttennmt.jpg
sun_adgrxgxkwzynobeq.jpg
sun_adisysjjfwkwrzgh.jpg
sun_adonhugrdeurtdfx.jpg
sun_adtlxvlfsrlbpcvu.jpg
sun_aetuswgzdkjpewoq.jpg
sun_aetynhnompeaihbx.jpg
sun_afgtgzodlwqwaint.jpg
sun_afjgnkyukz

5oooozidrptzw9po3n.jpg
5q6xvylwyg76201hf1.jpg
5qnfmo5ysyfot75tos.jpg
5qxshdvflygg9iyyv1.jpg
5rydeluvd3pper3nlu.jpg
5tvh05h3kvs7u0kdwr.jpg
5u3z9tt3f7laiv2ioh.jpg
5vwlcdxdulwec88cyb.jpg
5w3ne7fzpc5l9yjwoz.jpg
5wobm0uz4d0wqs0cvz.jpg
5wxkndw0jvuocg75av.jpg
5x78ur3jgvcy8gd05u.jpg
5ypzwlso6vkunp4rdh.jpg
5ywzm6ywhamcs6rsf7.jpg
5z017dan3ibcddygic.jpg
60kwv00307gchnlo7y.jpg
61bwku5m7n86sk4rpx.jpg
61vtity4vwkufukg5s.jpg
62sxsunamhmxbhb4po.jpg
63cpdzbm913mm38ql2.jpg
655qi9umbo1c4mkrm6.jpg
65xfi4im4fhxqspr0n.jpg
65z2mrwqer9u3iejwa.jpg
6774nq377pxuggw6ej.jpg
6aoyt36dch10n3e3rj.jpg
6de30w1x5hl7bstov0.jpg
6efcu37lrwrwqsu6of.jpg
6ejedlt6qmhhzkbxxa.jpg
6f1fbt2re1dlg08qea.jpg
6f6kcuetgku36zywvc.jpg
6ih3tuaipxycy3gpz4.jpg
6kt4xw4h2tpw47vfvo.jpg
6mn2fd0s3bui3cyq17.jpg
6mn2ofehifa7se0518.jpg
6p7nbe56s1r33d2iys.jpg
6qhpd937ssdbq67bp8.jpg
6qx7spkia23l58y14t.jpg
6s4lcm4i1ecm2uvufh.jpg
6w7es155u8lt65572d.jpg
6wa1pn3mbuz3v9ypnc.jpg
6wf6n9icgf19zt6ewp.jpg
6y0mrlswxszo1h4iu3.jpg
6zyacv8iwxtx07k7yz.jpg
72sb132jcc6

frame_01uj6751chwinefk.jpg
frame_04hrc06fj09lsgbl.jpg
frame_05ua6wgu5x0cw3c3.jpg
frame_06cc0rse9dti5b5s.jpg
frame_06cl39hbvcky126x.jpg
frame_07y33ggne7q7q8lm.jpg
frame_094180vevp9svfuy.jpg
frame_0baqzi64cd684ek3.jpg
frame_0cggc4ftnbe1b38l.jpg
frame_0cjawubg9ocitgiy.jpg
frame_0cwj1g4trfycew3a.jpg
frame_0e0fhiq9bsfkwino.jpg
frame_0edz6z2mjkgoi5xx.jpg
frame_0jk76tt78pxkjjnk.jpg
frame_0kxt219ei4sjqb9u.jpg
frame_0l0acozxi2uv3kbz.jpg
frame_0p00euagpk7lcmif.jpg
frame_0phgeao9lo2m3pmm.jpg
frame_0pkih1046jua1lji.jpg
frame_0pz7hl6lv28o6omr.jpg
frame_0q39cpis76hekfpo.jpg
frame_0r9x5g79iwn5ojv0.jpg
frame_0roy44fdq7y7wjtm.jpg
frame_0s85gyzlg136x1yh.jpg
frame_0sdmmwsijr25zfqz.jpg
frame_0tkny4hfy00l7t1n.jpg
frame_0wm81elmt7uk3b49.jpg
frame_0z5eyd6zs2h4gakm.jpg
frame_0z97e1dimfwxhufg.jpg
frame_113lf2mix1ce4ny4.jpg
frame_11nxo15sulazpyul.jpg
frame_124pcih721vz299l.jpg
frame_15cmwbi8as4rizqe.jpg
frame_16m97n6tksx6kd3g.jpg
frame_182ken2xwjnpw8sz.jpg
frame_1a5ttu4lhx5q8t03.jpg
frame_1azioz3k4p6d3f90.jpg
f

c:\users\juanl\appdata\local\programs\python\python38\lib\site-packages\PIL\TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


frame_26cxzwjb3jcyhpw2.jpg
frame_26opm62dub4njvuf.jpg
frame_275619rck8dyjgx3.jpg
frame_28cb2h5q65yd5pgg.jpg
frame_2aweuaa33bchghjj.jpg
frame_2clahdx0tsnz1zqr.jpg
frame_2d4exnp3o8xfapij.jpg
frame_2e3zu2yvy2x0iixv.jpg
frame_2gld39ud97tlrz2e.jpg
frame_2jeal5psm52yoizo.jpg
frame_2ke9i1m4bw60ck56.jpg
frame_2laobujpw91nskhz.jpg
frame_2mo6j8wqh55up4mz.jpg
frame_2n7n34ec5qzpi051.jpg
frame_2nabmrqqklvidkz6.jpg
frame_2nli2b3sozxtq9vy.jpg
frame_2ogcz0hif1vtap5h.jpg
frame_2qbggceh7ygnbb7o.jpg
frame_2r7gws111tc8mywg.jpg
frame_2scp6wswpye6wyq7.jpg
frame_2sphp8eiy62qunmm.jpg
frame_2sum3gmfcrjwlanz.jpg
frame_2tp52goo0558sg8f.jpg
frame_2tu6bphqgi2ajfur.jpg
frame_2uhxq3twd5heu5f3.jpg
frame_2uqob1wgn3x7rr5v.jpg
frame_2vdfmpubfzjbhjkj.jpg
frame_3023fbaqevnimxlm.jpg
frame_30bsqalnhumspiny.jpg
frame_34p274enlkogh5zr.jpg
frame_34r8ubhedth86y1l.jpg
frame_369c5ifgptcac061.jpg
frame_37ejzvdr6lw384m1.jpg
frame_37s0dd084rcxlq2b.jpg
frame_37zoy9thss5owgyu.jpg
frame_38ovx2pkxjx3ir62.jpg
frame_39meb1c6weiduxwg.jpg
f

frame_amn54iz0erjrqarl.jpg
frame_an4f7d0m1wmax0yo.jpg
frame_anddvlct8kvtyy61.jpg
frame_apnlzirxodrxi8ek.jpg
frame_arhwivnln58a04cu.jpg
frame_arqw4sa3kh4xyjg5.jpg
frame_asgxgr5n6owapuzl.jpg
frame_awa9w8ttgr7i8hi1.jpg
frame_awt54t67h1n86fj3.jpg
frame_ax5q7ziiwekqet5a.jpg
frame_axc2kw7c41653cbr.jpg
frame_axc2zi5i8woesyhb.jpg
frame_ayddf6r8v11nqym4.jpg
frame_ayg0ndulhw4p016l.jpg
frame_azgd808j15amdmoj.jpg
frame_b0dh1326mxb1ux4h.jpg
frame_b0patujetrnzqrqh.jpg
frame_b0zzw2hwqoxycebw.jpg
frame_b16zcr204faf283e.jpg
frame_b1pmip403ce5sc6w.jpg
frame_b28zz3foaps7zbc5.jpg
frame_b3jojfgv60r6nbys.jpg
frame_b48qxyiyrtm514dn.jpg
frame_b698q71xh6icsvnt.jpg
frame_b6mdottj69bl4r92.jpg
frame_b8pc8hqh97zafi04.jpg
frame_b9quvn3y5uvw0osh.jpg
frame_banhg5nf6zmt6xwb.jpg
frame_bbc7uw0gjxa7fnd6.jpg
frame_bdjzhh0vx3y6os5k.jpg
frame_be1ph4gcls3g33i6.jpg
frame_be3yivhd3v1obtiv.jpg
frame_beizpc8vgys9a8sl.jpg
frame_bf9wy0k3wof0bmlc.jpg
frame_bfp1cy5kr5rb36m2.jpg
frame_bgw5s5hdekqg29si.jpg
frame_bhgtm9w3msl0o92s.jpg
f

frame_l6amvvxfw48v7nkt.jpg
frame_l83odsb7wc9hzcbp.jpg
frame_l8w7vc3vlnov45x2.jpg
frame_l8xyzg1792swrbze.jpg
frame_l9hs07n313kb5tvn.jpg
frame_l9kt8eh7c2z4at1a.jpg
frame_la1cz93hl069a91u.jpg
frame_la7goju3agm64xag.jpg
frame_laq1jpvu120ige6u.jpg
frame_lbtaovms93hpq80s.jpg
frame_lct4xs5p1qju9cgn.jpg
frame_le6gog8y5al4orzf.jpg
frame_leckbk5wfmbwqbjb.jpg
frame_leryhdrgjeuhnjfi.jpg
frame_lew226oz7fq76rkt.jpg
frame_lfevypv62osee7y2.jpg
frame_li3xir3u76ka9aoz.jpg
frame_liaawqhu859i16r6(1).jpg
frame_lj4q1d94m187nmkc.jpg
frame_ljgqkx1nzix6t6ou.jpg
frame_ljuz71qmjsws7nnu.jpg
frame_lk95wqo8uwqg7ovk.jpg
frame_ll51978qso57jd9w.jpg
frame_llxv6akyv7nusd4m.jpg
frame_lm76u594ef3vqhho.jpg
frame_lmh569ghcs6efdm4.jpg
frame_lmr6c0ze0tsrk9dx.jpg
frame_ln2eivl3ya2hp2f9.jpg
frame_ln89za1sj3d024r4.jpg
frame_lnogvq26i86x4jcq.jpg
frame_lop5bje0degb6e6o.jpg
frame_lpvbq5tdoar0rdrv.jpg
frame_lpx9w3ibgw5f4ieb.jpg
frame_lq2fy5s4raestjex.jpg
frame_lqnkk7r1i4chwaxu.jpg
frame_lr3c1wfadtpu6des.jpg
frame_lsv2p5wkqns4p9uw.jp

COCO_train2014_000000580668.jpg
COCO_val2014_000000015032.jpg
COCO_val2014_000000020671.jpg
COCO_val2014_000000021989.jpg
COCO_val2014_000000025668.jpg
COCO_val2014_000000037655.jpg
COCO_val2014_000000039718.jpg
COCO_val2014_000000039731.jpg
COCO_val2014_000000040813.jpg
COCO_val2014_000000048917.jpg
COCO_val2014_000000061566.jpg
COCO_val2014_000000067881.jpg
COCO_val2014_000000070480.jpg
COCO_val2014_000000105220.jpg
COCO_val2014_000000105234.jpg
COCO_val2014_000000111644.jpg
COCO_val2014_000000113720.jpg
COCO_val2014_000000118106.jpg
COCO_val2014_000000121591.jpg
COCO_val2014_000000122476.jpg
COCO_val2014_000000124013.jpg
COCO_val2014_000000124629.jpg
COCO_val2014_000000131804.jpg
COCO_val2014_000000137654.jpg
COCO_val2014_000000140542.jpg
COCO_val2014_000000144269.jpg
COCO_val2014_000000145604.jpg
COCO_val2014_000000147173.jpg
COCO_val2014_000000147577.jpg
COCO_val2014_000000153717.jpg
COCO_val2014_000000168580.jpg
COCO_val2014_000000168781.jpg
COCO_val2014_000000173229.jpg
COCO_val

In [51]:
# load prepared planet dataset
from numpy import load
data = load('planet_data.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)

Loaded:  (7749, 128, 128, 3) (7749, 26)


In [52]:
# load train and test dataset
def load_dataset():
	# load dataset
	data = load('planet_data.npz')
	X, y = data['arr_0'], data['arr_1']
	# separate into train and test datasets
	trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
	return trainX, trainY, testX, testY

In [58]:
# test f-beta score
from numpy import load
from numpy import ones
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

# load train and test dataset
def load_dataset():
	# load dataset
	data = load('planet_data.npz')
	X, y = data['arr_0'], data['arr_1']
	# separate into train and test datasets
	trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
	print(trainX.shape, trainY.shape, testX.shape, testY.shape)
	return trainX, trainY, testX, testY

# load dataset
trainX, trainY, testX, testY = load_dataset()
# make all one predictions
train_yhat = asarray([ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
test_yhat = asarray([ones(testY.shape[1]) for _ in range(testY.shape[0])])
# evaluate predictions
train_score = fbeta_score(trainY, train_yhat, 2, average='samples')
test_score = fbeta_score(testY, test_yhat, 2, average='samples')
print('All Ones: train=%.3f, test=%.3f' % (train_score, test_score))

(5424, 128, 128, 3) (5424, 26) (2325, 128, 128, 3) (2325, 26)
All Ones: train=0.281, test=0.280


C:\Users\juanl\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\juanl\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [59]:
from keras import backend

# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

In [60]:
# load dataset
trainX, trainY, testX, testY = load_dataset()
# make all one predictions
train_yhat = asarray([ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
test_yhat = asarray([ones(testY.shape[1]) for _ in range(testY.shape[0])])
# evaluate predictions with sklearn
train_score = fbeta_score(trainY, train_yhat, 2, average='samples')
test_score = fbeta_score(testY, test_yhat, 2, average='samples')
print('All Ones (sklearn): train=%.3f, test=%.3f' % (train_score, test_score))
# evaluate predictions with keras
train_score = fbeta(backend.variable(trainY), backend.variable(train_yhat))
test_score = fbeta(backend.variable(testY), backend.variable(test_yhat))
print('All Ones (keras): train=%.3f, test=%.3f' % (train_score, test_score))

(5424, 128, 128, 3) (5424, 26) (2325, 128, 128, 3) (2325, 26)
All Ones (sklearn): train=0.281, test=0.280


C:\Users\juanl\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
C:\Users\juanl\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


All Ones (keras): train=0.281, test=0.280


In [65]:
# baseline model for the planet dataset
import sys
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

In [61]:
# define cnn model
def define_model(in_shape=(128, 128, 3), out_shape=17):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(out_shape, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
	return model

In [62]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Fbeta')
	pyplot.plot(history.history['fbeta'], color='blue', label='train')
	pyplot.plot(history.history['val_fbeta'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [63]:
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = datagen.flow(trainX, trainY, batch_size=128)
	test_it = datagen.flow(testX, testY, batch_size=128)
	# define model
	model = define_model()
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)
	# evaluate model
	loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))
	# learning curves
	summarize_diagnostics(history)

In [66]:
run_test_harness()

(5424, 128, 128, 3) (5424, 26) (2325, 128, 128, 3) (2325, 26)


c:\users\juanl\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


InvalidArgumentError:  Incompatible shapes: [128,26] vs. [128,17]
	 [[node binary_crossentropy/logistic_loss/mul (defined at <ipython-input-63-5a24e236cfcd>:13) ]] [Op:__inference_train_function_1195]

Function call stack:
train_function
